In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from math import sqrt
from sklearn.cross_validation import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
pd.options.display.float_format = '{:,.4f}'.format

plt.rc('figure', figsize=(10.0, 5.0))
matplotlib.rcParams.update({'font.size': 22})

# Make numpy outputs easier to read
np.set_printoptions(precision=3, formatter={'float': '{: 8.3f}'.format})

## Code for calculating DTW distance

In [3]:
def DTWDistance(s1, s2,w):
    DTW={}

    w = max(w, abs(len(s1)-len(s2)))

    for i in range(-1,len(s1)):
        for j in range(-1,len(s2)):
            DTW[(i, j)] = float('inf')
    DTW[(-1, -1)] = 0

    for i in range(len(s1)):
        for j in range(max(0, i-w), min(len(s2), i+w)):
            dist= (s1[i]-s2[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])

    return sqrt(DTW[len(s1)-1, len(s2)-1])

In [4]:
def LB_Keogh(s1,s2,r):
    LB_sum=0
    for ind,i in enumerate(s1):

        lower_bound=min(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
        upper_bound=max(s2[(ind-r if ind-r>=0 else 0):(ind+r)])

        if i>upper_bound:
            LB_sum=LB_sum+(i-upper_bound)**2
        elif i<lower_bound:
            LB_sum=LB_sum+(i-lower_bound)**2

    return sqrt(LB_sum)

In [26]:
def knnDTW(train,test, w):
    preds=[]
    for ind,i in enumerate(test):
        if(ind%1000==0):
            print(f'checking on {ind}th element')
        min_dist=float('inf')
        closest_seq=[]
        for j in train:
            if LB_Keogh(i[:-1],j[:-1],5)<min_dist:
                dist=DTWDistance(i[:-1],j[:-1],w)
                if dist<min_dist:
                    min_dist=dist
                    closest_seq=j
        preds.append(closest_seq[-1])
    return preds

In [6]:
with open('./data/processed/X_train.pkl', 'rb') as picklefile:
    X_train = pickle.load(picklefile)
with open('./data/processed/y_train.pkl', 'rb') as picklefile:
    y_train = pickle.load(picklefile)
with open('./data/processed/X_test.pkl', 'rb') as picklefile:
    X_test = pickle.load(picklefile)
with open('./data/processed/y_test.pkl', 'rb') as picklefile:
    y_test = pickle.load(picklefile)

In [ ]:
kf = KFold(n=train_ts.shape[0], n_folds=5, shuffle=True)
FP = []
FN = []
TP = []
TN = []

for train, test in kf:
    train_cv = X_train[train]
    test_cv = X_train[test]
    

In [19]:
topans = pd.concat([X_train.iloc[:, 29:41], y_train.to_frame()], axis=1)

In [21]:
train_ts, test_ts = train_test_split(topans.values, test_size=0.2, random_state=4444, stratify=topans['inactive'])

In [27]:
preds = knnDTW(train_ts, test_ts, 4)

checking on 0th element


KeyboardInterrupt: 

In [29]:
test_ts.shape

(44741, 13)